# **Data Collection**

## Objectives

* Fetch images from Kaggle and split into train, test and validation folders

## Inputs

* Kaggle JSON file - the authentication token.

## Outputs

* Generate Dataset: inputs/datasets/cats_vs_dogs_dataset

## Additional Comments

* Must delete unlabelled images from the Kaggle dataset as they can not be used for this project.


---

# Install packages

Install requirements and import libraries

In [ ]:
%pip install -r /workspace/pp5-cats-vs-dogs/requirements.txt 2>/dev/null | grep -v 'Requirement already satisfied'
print('Requirements installed.')
import os
import numpy
import zipfile

# Change working directory

Change working directory to root project folder

In [ ]:
current_dir = os.getcwd()
print('Current folder: ' + current_dir)
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
print('New folder: ' + current_dir)

---

# Download dataset

* Install Kaggle, configure the directory, and set permissions for the Kaggle authentication JSON.
* Download the Kaggle dataset.
* Unzip the file and delete the zip file and unlabeled images.

In [ ]:
%pip install kaggle==1.5.12 2>/dev/null | grep -v 'Requirement already satisfied'
print('Requirements installed.')

os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json
print('Directory configured and permissions set.')

DestinationFolder = 'inputs/cats_vs_dogs_dataset'
! kaggle competitions download -c dogs-vs-cats -p {DestinationFolder}

print('Extracting files...')
with zipfile.ZipFile(DestinationFolder + '/dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)    

with zipfile.ZipFile(DestinationFolder + '/train.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/dogs-vs-cats.zip')
os.remove(DestinationFolder + '/test1.zip')
os.remove(DestinationFolder + '/train.zip')
os.remove(DestinationFolder + '/sampleSubmission.csv')
print('Unused files deleted.')


---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---


# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)


# Conclusions and Next Steps